# Problem 3: Time-Invariant Kalman Filtering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/Quantlet_Problem03_KalmanFiltering.ipynb)

---

**QuantLet Name:** `SSM_KalmanFiltering_TimeInvariant`  
**Published in:** State Space Models and Markov Switching Models — Chapter 8  
**Author:** Daniel Traian Pele  
**Institution:** Bucharest University of Economic Studies  
**Date:** February 2026  

---

## Problem
Derive and implement the Kalman filtering, forecasting, and smoothing formulae for the time-invariant state space model.

---
## Plotting: ✅ Transparent background · ✅ No grid · ✅ Legend outside bottom


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'axes.grid': False,
    'font.size': 11, 'axes.labelsize': 12,
    'axes.titlesize': 13, 'figure.figsize': (12, 5)
})
print('Setup complete.')


## 1. Time-Invariant State Space Model

**Transition:** $\mathbf{X}_{t+1} = \mathbf{c} + \mathbf{F}\mathbf{X}_t + \mathbf{R}\boldsymbol{\eta}_t$

**Observation:** $\mathbf{Y}_t = \mathbf{d} + \mathbf{Z}\mathbf{X}_t + \boldsymbol{\epsilon}_t$

with $\boldsymbol{\eta}_t \sim N(0, \mathbf{Q})$ and $\boldsymbol{\epsilon}_t \sim N(0, \mathbf{H})$.


## 2. Kalman Filter Implementation from Scratch


In [ ]:
def kalman_filter(Y, F, Z, c, d, R, Q, H, X0, P0):
    """Kalman filter for time-invariant state space model."""
    T = len(Y)
    m = len(X0)
    
    # Storage
    X_pred = np.zeros((T, m))   # X_{t|t-1}
    P_pred = np.zeros((T, m, m))
    X_filt = np.zeros((T, m))   # X_{t|t}
    P_filt = np.zeros((T, m, m))
    innovations = np.zeros(T)
    
    X_prev = X0
    P_prev = P0
    
    for t in range(T):
        # Prediction step
        X_pred[t] = c + F @ X_prev
        P_pred[t] = F @ P_prev @ F.T + R @ Q @ R.T
        
        # Innovation
        nu = Y[t] - d - Z @ X_pred[t]
        innovations[t] = nu
        V = Z @ P_pred[t] @ Z.T + H
        
        # Kalman gain
        K = P_pred[t] @ Z.T / V
        
        # Update step (filtering)
        X_filt[t] = X_pred[t] + K.flatten() * nu
        P_filt[t] = P_pred[t] - np.outer(K.flatten(), Z @ P_pred[t])
        
        X_prev = X_filt[t]
        P_prev = P_filt[t]
    
    return X_pred, P_pred, X_filt, P_filt, innovations

print('Kalman filter function defined.')


In [ ]:
def kalman_smoother(X_pred, P_pred, X_filt, P_filt, F):
    """Rauch-Tung-Striebel (RTS) backward smoother."""
    T, m = X_filt.shape
    X_smooth = np.zeros_like(X_filt)
    P_smooth = np.zeros_like(P_filt)
    
    X_smooth[-1] = X_filt[-1]
    P_smooth[-1] = P_filt[-1]
    
    for t in range(T - 2, -1, -1):
        J = P_filt[t] @ F.T @ np.linalg.inv(P_pred[t + 1])
        X_smooth[t] = X_filt[t] + J @ (X_smooth[t + 1] - X_pred[t + 1])
        P_smooth[t] = P_filt[t] + J @ (P_smooth[t + 1] - P_pred[t + 1]) @ J.T
    
    return X_smooth, P_smooth

print('Kalman smoother function defined.')


## 3. Example: Local Level Model (Random Walk + Noise)

$$X_{t+1} = X_t + \eta_t, \quad Y_t = X_t + \varepsilon_t$$


In [ ]:
# Simulate local level model
np.random.seed(42)
T = 200
sigma_eta = 0.5   # state noise
sigma_eps = 1.0   # observation noise

# True states and observations
X_true = np.cumsum(np.random.normal(0, sigma_eta, T))
Y_obs = X_true + np.random.normal(0, sigma_eps, T)

# Kalman filter parameters
F = np.array([[1.0]])
Z = np.array([[1.0]])
c = np.array([0.0])
d = np.array([0.0])
R = np.array([[1.0]])
Q = np.array([[sigma_eta**2]])
H = np.array([[sigma_eps**2]])
X0 = np.array([0.0])
P0 = np.array([[1.0]])

# Run Kalman filter
X_pred, P_pred, X_filt, P_filt, innovations = kalman_filter(
    Y_obs, F, Z, c, d, R, Q, H, X0, P0)

# Run smoother
X_smooth, P_smooth = kalman_smoother(X_pred, P_pred, X_filt, P_filt, F)

print(f'Filter MSE:   {np.mean((X_filt[:, 0] - X_true)**2):.4f}')
print(f'Smoother MSE: {np.mean((X_smooth[:, 0] - X_true)**2):.4f}')


In [ ]:
# Plot: Filtering, Prediction, Smoothing
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
fig.patch.set_alpha(0)

titles = ['Kalman Prediction ($X_{t|t-1}$)',
          'Kalman Filtering ($X_{t|t}$)',
          'Kalman Smoothing ($X_{t|T}$)']
estimates = [X_pred[:, 0], X_filt[:, 0], X_smooth[:, 0]]
colors_est = ['darkorange', 'seagreen', 'crimson']

for ax, title, est, col in zip(axes, titles, estimates, colors_est):
    ax.patch.set_alpha(0); ax.grid(False)
    ax.scatter(range(T), Y_obs, s=5, alpha=0.3, color='grey', label='Observations $Y_t$')
    ax.plot(X_true, color='steelblue', linewidth=1.5, label='True state $X_t$')
    ax.plot(est, color=col, linewidth=1.2, linestyle='--', label=title)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_ylabel('Value')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13), ncol=3, frameon=False)

axes[-1].set_xlabel('Time')
plt.tight_layout(rect=[0, 0.03, 1, 1])
plt.savefig('Kalman_Filter_Smoother.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()


In [ ]:
# Confidence bands for smoother
fig, ax = plt.subplots(figsize=(14, 6))
fig.patch.set_alpha(0); ax.patch.set_alpha(0); ax.grid(False)

std_smooth = np.sqrt(P_smooth[:, 0, 0])
ax.fill_between(range(T),
                X_smooth[:, 0] - 1.96*std_smooth,
                X_smooth[:, 0] + 1.96*std_smooth,
                alpha=0.3, color='crimson', label='95% CI')
ax.scatter(range(T), Y_obs, s=5, alpha=0.3, color='grey', label='Observations')
ax.plot(X_true, color='steelblue', linewidth=1.5, label='True state')
ax.plot(X_smooth[:, 0], color='crimson', linewidth=1.2, label='Smoothed estimate')
ax.set_title('Kalman Smoother with 95% Confidence Bands', fontsize=13, fontweight='bold')
ax.set_xlabel('Time'); ax.set_ylabel('Value')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), ncol=4, frameon=False)
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig('Kalman_Smoother_CI.png', dpi=150, bbox_inches='tight', transparent=True)
plt.show()


## Conclusion

The Kalman filter, predictor, and smoother are implemented for the time-invariant case. The smoother yields the lowest MSE by using both past and future observations.
